# Análise de Dados de Filmes

Este notebook contém várias análises sobre dados de filmes, explorando diferentes aspectos como orçamento, receita, equipe, gêneros e muito mais.

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os

# Carregando o dataset
script_dir = os.path.dirname(os.path.abspath('__file__'))
DATASET_PATH = os.path.join(script_dir, 'movies.csv')

try:
    df = pd.read_csv(DATASET_PATH)
    print("Dataset carregado com sucesso!")
    print(f"Número de filmes: {len(df)}")
    print("\nPrimeiras linhas do dataset:")
    display(df.head())
except FileNotFoundError:
    print("Erro: Arquivo movies.csv não encontrado")

## 1. Análise de Orçamento e Receita

Analisando a relação entre o orçamento dos filmes e sua receita, calculando também o ROI (Return on Investment).

In [ ]:
def analise_orcamento_receita(df):
    df['ROI'] = (df['revenue'] - df['budget']) / df['budget']

    plt.figure(figsize=(10, 6))
    plt.scatter(df['budget'], df['revenue'], alpha=0.5)
    plt.xlabel('Orçamento')
    plt.ylabel('Receita')
    plt.title('Relação entre Orçamento e Receita')
    plt.show()

    correlacao = df['budget'].corr(df['revenue'])
    print(f"Correlação entre orçamento e receita: {correlacao:.2f}")
    
    print(f"\nEstatísticas do ROI:")
    print(df['ROI'].describe())

analise_orcamento_receita(df)

## 2. Análise de Produtoras

Analisando como o número de produtoras envolvidas se correlaciona com o orçamento e a bilheteria dos filmes.

In [ ]:
def analise_produtoras(df):
    print("Análise de produtoras em desenvolvimento...")
    df['production_company_count'] = df['production_companies'].apply(lambda x: len(eval(x)))

    company_analysis = df.groupby('production_company_count').agg({
        'budget': 'mean',
        'revenue': 'mean',
        'vote_average': 'mean'
    }).reset_index()

    plt.figure(figsize=(12, 6))
    plt.bar(company_analysis['production_company_count'], company_analysis['revenue'] / 1000000)
    plt.xlabel('Número de empresas')
    plt.ylabel('Faturamento médio (milhões de U$)')
    plt.title('Faturamento Médio por Número de Empresas Produtoras')

analise_produtoras(df)

## 3. Análise de Atores

Atores que aparecem mais de uma vez nos filmes mais bem avaliados (média de avaliação maior que 8.5)

In [ ]:
def analise_diretores_atores(df):
    df['vote_average'] = pd.to_numeric(df['vote_average'], errors='coerce')

    well_evaluated_threshold = 8.2
    high_rated_df = df[df['vote_average'] >= well_evaluated_threshold].copy()

    high_rated_df['cast'] = high_rated_df['cast'].astype(str)

    def split_cast(cast_str):
        tokens = cast_str.strip().split()
        return [' '.join(tokens[i:i+2]) for i in range(0, len(tokens), 2) if i+1 < len(tokens)]

    high_rated_df['cast_list'] = high_rated_df['cast'].apply(split_cast)
    exploded_cast = high_rated_df.explode('cast_list')

    actor_counts = exploded_cast['cast_list'].value_counts()
    popular_actors = actor_counts[actor_counts > 1]

    plt.figure(figsize=(12, 6))
    popular_actors.plot(kind='bar', color='skyblue')
    plt.title('Atores que apareceram com mais frequência em filmes bem avaliados')
    plt.xlabel('Ator')
    plt.ylabel('Frequência')
    plt.xticks(rotation=45, ha='right')
    plt.show()
    
analise_diretores_atores(df)


## 4. Análise de Duração e Popularidade

Vamos investigar se existe alguma relação entre a duração do filme e sua popularidade ou avaliação.

In [ ]:
def analise_duracao_popularidade(df):
    print("Análise de duração e popularidade em desenvolvimento...")
    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(15, 6))

    ax1.scatter(df['runtime'], df['popularity'])
    ax1.set_title('Duração do Filme vs. Popularidade')
    ax1.set_xlabel('Duração do filme (minutos)')
    ax1.set_ylabel('Popularidade')

    ax2.scatter(df['runtime'], df['vote_average'])
    ax2.set_title('Duração do Filme vs. Avaliação Média')
    ax2.set_xlabel('Duração do filme (minutos)')
    ax2.set_ylabel('Avaliação média')

    runtime_popularity_corr = df['runtime'].corr(df['popularity'])
    runtime_vote_corr = df['runtime'].corr(df['vote_average'])
    print(f"Duração do filme - Correlação com popularidade: {runtime_popularity_corr:.2f}")
    print(f"Duração do filme - Correlação com avaliação média: {runtime_vote_corr:.2f}") 

analise_duracao_popularidade(df)

## 5. Análise de Diversidade de Gênero

Vamos analisar a diversidade de departamento nas equipes de produção dos filmes.

In [ ]:
def analise_diversidade_departamento(df):
    print("Análise de diversidade de departamento em desenvolvimento...")

    def extract_department_data(crew_str):
        crew = eval(crew_str)
        dept_counts = {}
        
        for person in crew:
            dept = person.get('department', 'Unknown')
            dept_counts[dept] = dept_counts.get(dept, 0) + 1
        
        return pd.Series(dept_counts)

    department_df = df['crew'].apply(extract_department_data)
    department_totals = department_df.sum().sort_values(ascending=False)

    plt.figure(figsize=(10, 6))
    department_totals.plot(kind='bar')
    plt.ylabel('Número de Pessoas')
    plt.xlabel('Departamento')
    plt.title('Distribuição por Departamento em Equipes de Filmes')
    plt.xticks(rotation=45)
    plt.tight_layout()
    plt.show()
    
analise_diversidade_departamento(df)

## 6. Análise de Gêneros e Orçamento

Vamos investigar quais gêneros têm maiores orçamentos e se isso resulta em maior receita.

In [ ]:
def analise_generos_orcamento(df):
    df['genre_list'] = df['genres'].astype(str).str.split()

    exploded_df = df.explode('genre_list')

    genre_analysis = exploded_df.groupby('genre_list').agg({
        'budget': 'mean',
        'revenue': 'mean',
        'vote_average': 'mean',
        'index': 'count'
    }).rename(columns={'index': 'count'}).reset_index()

    genre_analysis = genre_analysis.sort_values('revenue', ascending=False)

    genres = genre_analysis['genre_list']
    x = np.arange(len(genres))
    width = 0.35

    _, ax = plt.subplots(figsize=(14, 8))
    ax.bar(x - width/2, genre_analysis['revenue'] / 1e6, width, label='Faturamento (M)')
    ax.bar(x + width/2, genre_analysis['budget'] / 1e6, width, label='Orçamento (M)', color='orange')

    ax.set_xlabel('Gênero')
    ax.set_ylabel('Quantidade (em milhões de $USD)')
    ax.set_title('Média de Faturamento e Orçamento por Gênero')
    ax.set_xticks(x)
    ax.set_xticklabels(genres, rotation=45, ha='right')
    ax.legend()
    ax.grid(axis='y', linestyle='--', alpha=0.7)

    plt.tight_layout()
    plt.show()

analise_generos_orcamento(df)

## 7. Análise de Tamanho da Equipe

Vamos analisar como o tamanho da equipe se relaciona com o orçamento e o sucesso dos filmes.

In [ ]:
def analise_equipe_orcamento(df):
    df['crew_size'] = df['crew'].apply(lambda x: len(eval(x)))

    plt.figure(figsize=(10, 6))
    plt.scatter(df['crew_size'], df['revenue'], alpha=0.7)
    plt.title('Tamanho da Equipe vs. Faturamento')
    plt.xlabel('Tamanho da equipe')
    plt.ylabel('Faturamento (R$)')
analise_equipe_orcamento(df)

## 8. Análise dos filmes em relação aos diretores

Vamos analisar quais foram os diretores com mais filmes lançados e sua média de avaliação

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt

def analise_filmes_diretores(df):
    df['vote_average'] = pd.to_numeric(df['vote_average'], errors='coerce')

    df = df[df['director'].notna()]

    director_stats = df.groupby('director').agg(
        movie_count=('title', 'count'),
        average_rating=('vote_average', 'mean')
    )
    top_director = director_stats.sort_values(by='movie_count', ascending=False).head(1)

    print("Diretor com mais filmes:")
    print(top_director)

    plt.figure(figsize=(12, 6))
    plt.scatter(director_stats['movie_count'], director_stats['average_rating'], alpha=0.6, color='teal')
    plt.title('Diretores: Número de filmes vs Avaliação média')
    plt.xlabel('Número de Filmes')
    plt.ylabel('Avaliação Média')
    plt.tight_layout()
    plt.show()

analise_filmes_diretores(df)


## 9. Análise de Data de Lançamento

Vamos investigar se existe correlação entre a data de lançamento e o desempenho de bilheteria.

In [ ]:
def analise_data_bilheteria(df):
    print("Análise de data de lançamento em desenvolvimento...")
    language_analysis = df.groupby('original_language').agg({
        'index': 'count',
        'budget': 'mean',
        'revenue': 'mean',
        'vote_average': 'mean'
    }).rename(columns={'index': 'movie_count'}).reset_index()

    language_analysis = language_analysis[language_analysis['movie_count'] >= 2]

    language_analysis = language_analysis.sort_values('revenue', ascending=False)

    plt.figure(figsize=(12, 6))
    plt.bar(language_analysis['original_language'], language_analysis['revenue'] / 1000000)
    plt.ylabel('Receita Média (milhões $)')
    plt.xlabel('Idioma Original')
    plt.title('Receita Média por Idioma Original')
analise_data_bilheteria(df)

## 10. Análise do rendimento em relação ao mês de lançamento.

Analisando como o mês de lançamento do filme influencia em seu rendimento

In [ ]:
def analise_orcamento_tempo(df):
    df['release_month'] = pd.to_datetime(df['release_date']).dt.month

    monthly_performance = df.groupby('release_month').agg({
        'revenue': 'mean',
        'index': 'count'
    }).rename(columns={'index': 'movie_count'}).reset_index()

    plt.figure(figsize=(12, 6))
    plt.bar(monthly_performance['release_month'], monthly_performance['revenue'] / 1000000)
    plt.xlabel('Mês de Lançamento')
    plt.ylabel('Receita Média (milhões $)')
    plt.title('Receita Média por Mês de Lançamento')
    plt.xticks(range(1, 13), ['Jan', 'Fev', 'Mar', 'Abr', 'Mai', 'Jun', 'Jul', 'Ago', 'Set', 'Out', 'Nov', 'Dez'])
analise_orcamento_tempo(df)